In [1]:
from qiskit import *
from qiskit import Aer, transpile
import qiskit.quantum_info as qi
from qiskit.visualization import plot_histogram, plot_bloch_vector, plot_bloch_multivector
from qiskit.providers.ibmq import least_busy
from qiskit.extensions import Initialize
import qiskit.tools.jupyter

from math import sqrt, pi
from latex import *
%matplotlib inline
%load_ext tikzmagic

I think I can safely say that nobody understands quantum mechanics … Do not keep saying to yourself, if you can possibly avoid it, “But how can it be like that?” because you will get “down the drain”, into a blind alley from which nobody has yet escaped. Nobody knows how it can be like that. – Richard Feynman

 * Computers and aliens
 * Compute to find out something about the world

<img src="https://quantum.country/assets/Africa_Europe_by_land.jpg" />

From [[1]](#references)

<img src="https://quantum.country/assets/Africa_Europe_by_sea.jpg" />

Quantum Computers - advantage over classical computers - can efficiently simulate quantium mechanical processes

# Short intro to Quantum Computing by Daniel Steger

* A quick-and-dirty intro Qubits 
* Quantum circuits
* Quantum teleportation
* Grover's algorithm
* Execution on IBM's quantum computer

# Qubits

## Base states [[2]](#references)

$$
\begin{align}
  |0 \rangle \equiv \begin{bmatrix} 1\\ 0 \end{bmatrix} \qquad  |1 \rangle \equiv \begin{bmatrix} 0\\ 1 \end{bmatrix}
\end{align}
$$

## Wavefunction for a generic quantum state

$$
\begin{align}
  |\phi \rangle = \alpha |0 \rangle + \beta |1 \rangle \qquad \alpha, \beta \in \mathbb{C} 
\end{align}
$$

$$
\begin{align}  
  \langle \phi | \phi \rangle = \begin{bmatrix} \alpha & \beta \end{bmatrix} \begin{bmatrix} \alpha\\ \beta \end{bmatrix} = \alpha^2 + \beta^2 = 1
\end{align}
$$
$$
\begin{align}
Norm: \Arrowvert \langle \phi | \phi \rangle\Arrowvert  = \sqrt{\alpha^2 + \beta^2} = 1
\end{align}
$$

In [ ]:
qc = QuantumCircuit(1)           # Create a quantum circuit with one qubit
initial_state = [0,1]            # Define initial_state as |1>
qc.initialize(initial_state, 0)  # Apply initialisation operation to the 0th qubit
qc.draw('mpl', scale=2)          # Let's view our circuit

## Init and run a state vector simulator

In [ ]:
statevec_sim = Aer.get_backend("statevector_simulator")
job = execute(qc, backend=statevec_sim)
result = job.result()
statevec = result.get_statevector()
display(Math(vector_to_latex(statevec, pretext="|\\psi\\rangle =")))

In [ ]:
qasm_sim = Aer.get_backend("qasm_simulator")
qc.measure_all()                # Measure all qubits
qasm_job = execute(qc, backend=qasm_sim, shots=1024).result()
counts = qasm_job.get_counts()
plot_histogram(counts)

# Interesting qubits

$$
|+\rangle = \tfrac{(|0\rangle + |1\rangle)}{\sqrt{2}} \qquad |-\rangle = \tfrac{(|0\rangle - |1\rangle)}{\sqrt{2}}
$$

In [ ]:
qc = QuantumCircuit(1)
initial_state = [1/sqrt(2), 1j/sqrt(2)]
qc.initialize(initial_state, 0)
qc.measure_all()
qc.draw('mpl', scale=2)

In [ ]:
qasm_sim = Aer.get_backend("qasm_simulator")
qasm_job = execute(qc, backend=qasm_sim, shots=1024).result()
counts = qasm_job.get_counts()
plot_histogram(counts)

# Bloch sphere

In [ ]:
from kaleidoscope.interactive import bloch_sphere

def toBloch(matrix):
    [[a, b], [c, d]] = matrix
    x = complex(c + b).real
    y = complex(c - b).imag
    z = complex(a - d).real
    return x, y, z

qc = QuantumCircuit(1)

rho = qi.DensityMatrix.from_instruction(qc)
bloch_sphere(toBloch(rho.data), vectors_annotation=True)

# Qubit gates

## Pauli gates

$$
\begin{align}
X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix} \qquad  Y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix} \qquad 
Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix} 
\end{align}
$$

### Applying a gate to a qubit

$$
\begin{align}
 X | 1\rangle = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix} \begin{bmatrix} 0\\ 1 \end{bmatrix} = \begin{bmatrix} 1\\ 0 \end{bmatrix}
\end{align}
$$

In [ ]:
qc = QuantumCircuit(1)           
initial_state = [0,1]           
qc.initialize(initial_state, 0)
qc.x(0)
qc.x(0)
qc.measure_all()            
qc.draw('mpl', scale=2)

In [ ]:
qasm_sim = Aer.get_backend("qasm_simulator")
job = execute(qc, backend=statevec_sim)
result = job.result()
statevec = result.get_statevector()
display(Math(vector_to_latex(statevec, pretext="|\\psi\\rangle =")))

## Hadamard gate
$$
\begin{align}
X = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
\end{align}
$$

In [ ]:
qasm_sim = Aer.get_backend("qasm_simulator")
qasm_job = execute(qc, backend=qasm_sim, shots=1024).result()
counts = qasm_job.get_counts()
plot_histogram(counts)

In [ ]:
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()
qc.draw('mpl', scale=2)

## CNOT Gate

| $c_{in}$ | $t_{in}$ | $c_{out}$ | $t_{out}$ |
|-------:|-------:|--------:|--------:|
| $|0\rangle$ | $|0\rangle$ | $|0\rangle$ | $|0\rangle$ |
| $|0\rangle$ | $|1\rangle$ | $|0\rangle$ | $|1\rangle$ |
| $|1\rangle$ | $|0\rangle$ | $|1\rangle$ | $\mathbf{|1\rangle}$ |
| $|1\rangle$ | $|1\rangle$ | $|1\rangle$ | $\mathbf{|0\rangle}$ |



In [ ]:
c = QuantumRegister(1, 'c')
t = QuantumRegister(1, 't')
qc = QuantumCircuit(c, t)
qc.cx(c, t)
qc.draw('mpl', scale=2)

In [ ]:
c = QuantumRegister(1, 'c')
t = QuantumRegister(1, 't')
qc = QuantumCircuit(c, t)
qc.h(c)
qc.cx(c, t)
qc.draw('mpl', scale=2) 

In [ ]:
job = execute(qc, backend=statevec_sim)
result = job.result()
statevec = result.get_statevector()
display(Math(vector_to_latex(statevec, pretext="|\\psi\\rangle =")))

## Bell state (EPR Pair)
$$
CNOT|+0\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}} = |\Phi^+\rangle
$$

In [ ]:
c = QuantumRegister(1, 'c')
t = QuantumRegister(1, 't')
qc = QuantumCircuit(c, t)
qc.h(c)
qc.h(t)
qc.z(t)
qc.cx(c, t)
qc.draw('mpl', scale=2) 

In [ ]:
statevec_sim = Aer.get_backend("statevector_simulator")
job = execute(qc, backend=statevec_sim)
result = job.result()
statevec = result.get_statevector()
display(Math(vector_to_latex(statevec, pretext="|\\psi\\rangle =")))

$$
CNOT|+-\rangle = \frac{|00\rangle - |01\rangle - |10\rangle + |11\rangle}{2} = |--\rangle
$$

## Toffoli Gate

| $c_{in1}$ | $c_{in1}$ | $t_{in}$ | $c_{out1}$ | $c_{out2}$ |$t_{out}$ |
|----------:|----------:|---------:|-----------:|-----------:|---------:|
| $|0\rangle$ | $|0\rangle$ | $|0\rangle$ | $|0\rangle$ | $|0\rangle$ | $|0\rangle$ |
| $|0\rangle$ | $|1\rangle$ | $|0\rangle$ | $|0\rangle$ | $|1\rangle$ | $|0\rangle$ |
| $|1\rangle$ | $|0\rangle$ | $|0\rangle$ | $|1\rangle$ | $|0\rangle$ | $|0\rangle$ |
| $|1\rangle$ | $|1\rangle$ | $|0\rangle$ | $|1\rangle$ | $|1\rangle$ | $\mathbf{|1\rangle}$ |

In [ ]:
c = QuantumRegister(2, 'c')
t = QuantumRegister(1, 't')
qc = QuantumCircuit(c, t)
qc.ccx(c[0], c[1], t)
qc.draw('mpl', scale=2)

# Quantum teleportation

Alice shared an EPR pair with Bob some time ago. Bob is in hiding and Alice got the mission to send a secret message in form of a qubit $ |\phi \rangle = \alpha |0 \rangle + \beta |1 \rangle$ to Bob.

She cannot copy the qubit like a classical bit. She cannot know the internal state of $|\psi\rangle$. The only thing she can do is send classical information.

## Setup

In [ ]:
# Protocol uses 3 qubits and 2 classical bits in 2 different registers
qr = QuantumRegister(3, name="q")
crz, crx = ClassicalRegister(1, name="crz"), ClassicalRegister(1, name="crx")
cresult = ClassicalRegister(1, name="cresult")
teleportation_circuit = QuantumCircuit(qr, crz, crx, cresult)

## Step 1 

Create the EPR pair in qubits $|q_1\rangle$ and $|q_2\rangle$.

In [ ]:
teleportation_circuit.h(qr[1])
teleportation_circuit.cx(qr[1], qr[2])
# And view the circuit so far:
teleportation_circuit.draw('mpl', scale=2)

## Step 2

Alice interacts with $|\psi\rangle$ and her half EPR.

In [ ]:
psi = [1/sqrt(2), 1/sqrt(2)]           
init_gate = Initialize(psi)
init_gate.label = "init"

teleportation_circuit.append(init_gate, [qr[0]])

teleportation_circuit.barrier()
teleportation_circuit.cx(qr[0], qr[1])
teleportation_circuit.h(qr[0])
teleportation_circuit.draw('mpl', scale=1.5)

## Step 3

Alice measures $|q_0\rangle$ and $|q_1\rangle$ and sends the infos to Bob.

In [ ]:
teleportation_circuit.barrier()
teleportation_circuit.measure(qr[0], crz)
teleportation_circuit.measure(qr[1], crx)
teleportation_circuit.draw('mpl', scale=1.5)

## Step 4

Bob applies the classical bits in 4 different ways:
* $00$ $\rightarrow$  Do nothing
* $01$ $\rightarrow$  Apply $X$-Gate
* $10$ $\rightarrow$  Apply $Z$-Gate
* $11$ $\rightarrow$  Apply $ZX$-Gates

In [ ]:
teleportation_circuit.barrier()
teleportation_circuit.x(qr[2]).c_if(crx, 1) # Apply gates if the registers 
teleportation_circuit.z(qr[2]).c_if(crz, 1) # are in the state '1'
teleportation_circuit.draw('mpl', scale=1.5)

In [ ]:
deentangle = init_gate.gates_to_uncompute()
teleportation_circuit.append(deentangle, [qr[2]])
teleportation_circuit.measure(qr[2], cresult)
teleportation_circuit.draw('mpl', scale=1.2)

In [ ]:
qasm_sim = Aer.get_backend("qasm_simulator")
qasm_job = execute(teleportation_circuit, backend=qasm_sim, shots=1024).result()
counts = qasm_job.get_counts()
plot_histogram(counts)

## Grover's algorithm [[3]](#references)

Search in an unsorted database. An oracle tells us if we have found the respective result:
$$
O|x\rangle = \begin{cases}
   &|x\rangle & x \neq w\\
   -&|x\rangle & x = w
    \end{cases}
$$

1. Init all qubits by applying a Hadamard gate

![Grover 1](grover1.png "image Title")

2. Apply the oracle $O$

![Grover 2](grover2.png "image Title")

3. Diffuse by reflecting $U = 2|s\rangle \langle s| - I$ with Hadamard gates

![Grover 3](grover3.png "image Title")


Apply the grover iteration about $\frac{\pi}{4}\sqrt{N}$ times and measure the output.

In [ ]:
%%tikz -s 1096,400 -f png -l arrows,calc
    \draw[very thick, ->, line join=miter,<->] (2.5,0) node(xline)[right] {$|s'\rangle$} -|
                    (0,2.5) node(yline)[above] {$|w\rangle$};
                    
    \draw[->, very thick, blue] (0,0) coordinate (es) -- ([turn]110:2.5) coordinate (ee) node[right]{$|s\rangle = |\psi_0\rangle$};
    \draw[thick, black] (1,0) arc [start angle=0, end angle=20, radius=1cm] node[left, xshift=0.1cm, yshift=-.2cm]{$\theta$}; 
            
    \begin{scope}[xshift=5cm]
        \draw[very thick, ->, line join=miter,<->] (0,2.5) node(eyline)[above] {$Amplitude$} |-
                        (2.5,0) node(exline)[right] {$Items$};

		\foreach \i in {0, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.8, 2, 2.2}
        	%\filldraw [line width=0.25mm,fill=blue!25] (\i,0) -- +(.1, 0) -- +(.1,1) -- +(\i,1);
            \draw[fill=blue!25!white] (\i,0) rectangle +(.2,1);
	
        \draw (2.4,1) node[right] {$\frac{1}{\sqrt{N}}$};  
		\draw[fill=blue!25!black] (1.6,0)node[below] {$w$} rectangle +(.2,1) ;
      \end{scope}

In [ ]:
%%tikz -s 1096,600 -f png -l arrows,calc
    \draw[very thick, ->, line join=miter,<->] (2.5,0) node(xline)[right] {$|s'\rangle$} -|
                    (0,2.5) node(yline)[above] {$|w\rangle$};
                    
    \draw[->, very thick, black] (0,0) coordinate (es) -- ([turn]110:2.5) coordinate (ee) node[right]{$|s\rangle$};
    \draw[->, very thick, blue] (0,0) coordinate (es) -- ([turn]70:2.5) coordinate (ee) node[right]{$O|s\rangle$};
    
    \draw[thick, black] (1,0) arc [start angle=0, end angle=-20, radius=1cm] node[left, xshift=.1cm, yshift=.2cm]{$\theta$}; 
    
    \draw[thick, dashed, black, ->] (2,0.7) arc [start angle=10, end angle=-30, radius=2cm]; 
    
    \begin{scope}[xshift=5cm]
        \draw[very thick, ->, line join=miter,<->] (0,2.5) node(eyline)[above] {$Amplitude$} |-
                        (2.5,0) node(exline)[right] {$Items$};

		\foreach \i in {0, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.8, 2, 2.2}
        	%\filldraw [line width=0.25mm,fill=blue!25] (\i,0) -- +(.1, 0) -- +(.1,1) -- +(\i,1);
        	\draw[fill=blue!25!white] (\i,0) rectangle +(.2,1);
        
        \draw (2.4,1) node[right] {$\leq\frac{1}{\sqrt{N}}$}; 
		\draw[fill=blue!25!black] (1.6,0) rectangle +(.2,-1) node[below] {$|w\rangle$};
      \end{scope}

In [ ]:
%%tikz -s 1024,600 -f png -l arrows,calc	
	\draw[very thick, ->, line join=miter,<->] (2.5,0) node(xline)[right] {$|s'\rangle$} -|
                    (0,2.5) node(yline)[above] {$|w\rangle$};
                    
    \draw[->, very thick, black] (0,0) coordinate (es) -- ([turn]110:2.5) coordinate (ee) node[right]{$|s\rangle = |\psi_0\rangle$};
    \draw[->, very thick, black] (0,0) coordinate (es) -- ([turn]70:2.5) coordinate (ee) node[right]{$O|s\rangle$};
    \draw[->, very thick, blue] (0,0) coordinate (es) -- ([turn]150:2.5) coordinate (ee) node[right]{$UO|s\rangle$};
    
    \draw[thick, black] (.93cm,.34cm) node[left, xshift=-0.1cm, yshift=0.1cm]{$2\theta$} arc [start angle=20, end angle=60, radius=1cm]; 
                
    \draw[thick, dashed, black, ->] (1.9,-0.7) arc [start angle=-30, end angle=60, radius=2cm];
    \begin{scope}[xshift=5cm]
        \draw[very thick, ->, line join=miter,<->] (0,2.5) node(eyline)[above] {$Amplitude$} |-
                        (2.5,0) node(exline)[right] {$Items$};

		\foreach \i in {0, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.8, 2, 2.2}
        	%\filldraw [line width=0.25mm,fill=blue!25] (\i,0) -- +(.1, 0) -- +(.1,1) -- +(\i,1);
        	\draw[fill=blue!25!white] (\i,0) rectangle +(.2,1);
	
		\draw[fill=blue!25!black] (1.6,0)node[below, xshift=.2cm] {$|w\rangle$} rectangle +(.2,1.5) ;
      \end{scope}
      

In [ ]:
def initialize_s(qc, qubits):
    """Apply a H-gate to 'qubits' in qc"""
    for q in qubits:
        qc.h(q)
    return qc

## Task: find a 2-bit combination which are palindromes:

States:
 * V_0, V_1, V_2

Conditions:
 * V_0 = V_2
 * V_1 = 1

## Building a XOR gate and negate it

In [ ]:
def NOTXOR(qc, a, b, output):
    qc.cx(a, output)
    qc.cx(b, output)
    qc.x(output)

In [ ]:
qc = QuantumCircuit(3)
NOTXOR(qc, 0, 1, 2)
qc.draw('mpl', scale=2)

## Building the Oracle $O$

In [ ]:
var_qubits = QuantumRegister(3, name='v')     # variable bits
clause_qubits = QuantumRegister(1, name='c')  # bits to store clause-checks
output_qubit = QuantumRegister(1, name='out')
cbits = ClassicalRegister(3, name='cbits')

# Create quantum circuit
qc = QuantumCircuit(var_qubits, clause_qubits, output_qubit, cbits)

def pali_oracle(qc, clause_qubits):
    # Compute clauses
    NOTXOR(qc, var_qubits[0], var_qubits[2], clause_qubits[0])

    # Flip 'output' bit if all clauses are satisfied
    qc.ccx(clause_qubits[0], var_qubits[1], output_qubit)

    # Uncompute clauses to reset clause-checking bits to 0
    NOTXOR(qc, var_qubits[0], var_qubits[2], clause_qubits[0])

pali_oracle(qc, clause_qubits)
qc.draw('mpl', scale=1.5)

## Building the Diffusor $U$

In [ ]:
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U"
    return U_s

#diffuser(3).draw('mpl', scale=1.5)

## Putting the algorithm together

In [ ]:
var_qubits = QuantumRegister(3, name='v')
clause_qubits = QuantumRegister(1, name='c')
output_qubit = QuantumRegister(1, name='out')
cbits = ClassicalRegister(3, name='cbits')
qc = QuantumCircuit(var_qubits, clause_qubits, output_qubit, cbits)

# Initialize 'out0' in state |-> or do HX
qc.initialize([1, -1]/np.sqrt(2), output_qubit)

# Initialize qubits in state |s>
qc.h(var_qubits)
qc.barrier()  # for visual separation

## First Iteration
# Apply our oracle
pali_oracle(qc, clause_qubits)
qc.barrier()  # for visual separation
# Apply our diffuser
qc.append(diffuser(3), [0,1,2])

## Second Iteration
#pali_oracle(qc, clause_qubits)
#qc.barrier()  # for visual separation
# Apply our diffuser
#qc.append(diffuser(3), [0,1,2])

# Measure the variable qubits
qc.measure(var_qubits, cbits)

qc.draw('mpl', fold=-1)

In [ ]:
qasm_sim = Aer.get_backend("qasm_simulator")
qasm_job = execute(qc, backend=qasm_sim, shots=1024).result()
counts = qasm_job.get_counts()
plot_histogram(counts)

# Execution on a real quantum computer

In [ ]:
# Load IBM provider
provider = IBMQ.load_account()
provider = IBMQ.get_provider("ibm-q")
provider.backends()

In [ ]:
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 5 and 
                                   not x.configuration().simulator and x.status().operational==True))
print("least busy backend: ", backend)

In [ ]:
backend = provider.get_backend('ibmq_quito')
backend

In [ ]:
coupling_map = backend.configuration().coupling_map
print(coupling_map)

In [ ]:
layout = {0: var_qubits[0], 1: clause_qubits[0], 2: var_qubits[2], 3: var_qubits[1], 4: output_qubit[0]}

In [ ]:
# Run our circuit on the least busy backend. Monitor the execution of the job in the queue
from qiskit.tools.monitor import job_monitor
transpiled_grover_circuit = transpile(qc, backend, initial_layout=layout, optimization_level=3)
transpiled_grover_circuit.draw('mpl', fold=1)

In [ ]:
job = backend.run(transpiled_grover_circuit, shots=8192)
job_monitor(job, interval=2)

In [ ]:
# Get the results from the computation
results = job.result()
answer = results.get_counts(qc)
plot_histogram(answer)

# Summary

<img src="https://quantum.country/assets/strange_loop.png" />

From [[1]](#references)

<a id='references'></a>
# Bibliography 

1. Andy Matuschak and Michael A. Nielsen, “Quantum Computing for the Very Curious”, (https://quantum.country/qcvc), San Francisco (2019).
2. Qiskit, “Representing Qubit States” (https://qiskit.org/textbook/ch-states/representing-qubit-states.html), IBM 2021.
3. Qiskit, “Grover's algorithm” (https://qiskit.org/textbook/ch-algorithms/grover.html), IBM 2021.
4. Michael A. Nielsen & Isaac L. Chuang, “[Quantum Computation and Quantum Information](https://en.wikipedia.org/wiki/Quantum_Computation_and_Quantum_Information)", Cambridge 2010.
5. M. A. Nielsen, E. Knill, R. Laflamme, “Complete quantum teleportation using nuclear magnetic resonance”, (https://arxiv.org/abs/quant-ph/9811020), Nature vol 395, 5 November 1998.

6. CS 269Q: Stanford Course on Quantum Computing, (https://cs269q.stanford.edu/index.html), 2019
7. Philip Krantz, Morten Kjaergaard, Fei Yan, Terry P. Orlando, Simon Gustavsson, William D. Oliver, “A Quantum Engineer's Guide to Superconducting Qubits”, (https://arxiv.org/abs/1904.06560), Applied Physics Reviews 6, 021318 (2019)
8. Qiskit Learning, (https://qiskit.org/learn/), IBM 2021
9. Andy Matuschak and Michael A. Nielsen, “Quantum Mechanics Distilled”, (https://quantum.country/qm), San Francisco (2020).
10. J. J. Sakurai, Jim Napolitano, “Modern Quantum Mechanics”, (https://doi.org/10.1017/9781108499996), Cambridge 2018.
11. Von Aristoteles zur Stringtheorie, (https://www.youtube.com/playlist?list=PLmDf0YliVUvGGAE-3CbIEoJM3DJHAaRzj)